Tutorial: Starting a NASim Environment: https://networkattacksimulator.readthedocs.io/en/latest/tutorials/loading.html#environment-settings


The three optional arguments control the environment modes:
- fully_obs: The observability mode of environment, if True then uses fully observable mode, otherwise is partially observable (default=False)
- flat_actions: If true then uses a flat action space, otherwise will uses a parameterised action space (default=True).
- flat_obs: If true then uses a 1D observation space, otherwise uses a 2D observation space (default=True)

In [2]:
import numpy
print(numpy.__version__) #updated to 1.23.4 > 1.18

import gymnasium
print(gymnasium.__version__) #updated to 0.28.1 > 0.17

#pyYaml is version 6.0

import networkx
print(networkx.__version__) #updated to 2.8.7 > 2.4

import prettytable
print(prettytable.__version__) #updated to 3.7.0 > 0.7.2

import matplotlib
print(matplotlib.__version__) #updated to 3.7.1 (shown in console with pip show matplotlib) > 3.1.3



1.24.3
0.28.1
3.1
3.7.0
3.7.1


In [3]:
#Making an existing scenario

import gymnasium

import nasim
env = nasim.make_benchmark("tiny")
#env1 = nasim.make_benchmark("tiny", rendermode="human") --> Couldn't get to work

#Loading a scenario from a YAML file
env2 = nasim.load("/Users/jacobalbright/Documents/GitHub/JacobPersonalTest/small.yaml")

#To generate a new environment with 5 hosts running a possible 3 services:
env3 = nasim.generate(5, 3)

# pass in some other parameters (say the number of possible operating systems) can be passed in as keyword arguments:
env4 = nasim.generate(5, 3, num_os=3)




Starting using OpenAI Gymnasium
- always import gymnasium as gym

- naming convention when using gymnasium.make(): ScenarioName[PO][2D][VA]-vX
    - ScenarioName is the name of the benchmark scenario in Camel Casing
    - [PO] is optional and included if environment is in partially observable mode, otherwise fully observable mode if not included.
    - [2D] is optional and included if environment is to return 2D observations, otherwise the environment returns 1D observations.
    - [VA] is optional and specifies the environment is to accept Vector actions (parametrised actions), if it is not included the environment expects integer (flat) actions.
    - vX is the environment version. Currently (as of version 0.10.0) all environments are on v0 (ALWAYS v0???)

Default: Tiny-v0 means the ‘tiny’ benchmark scenario in fully observable mode with flat action-space and flat observation space

Alternate: MediumPO2DVA-v0 means the ‘medium’ benchmark scenario in partially observable mode with parametrised action-space and 2D observation-space

In [16]:
#testing rendering

import gymnasium as gym
env5 = gym.make("nasim:Tiny-v0")

# to specify render mode
env6 = gym.make("nasim:TinyPO-v0", render_mode="human")

env6.reset()
# render the environment
# (if render_mode="human" is not passed during initialization this will do nothing)
env6.render()

Observation:
+---------+------------------+------------------+-----------------+
| Success | Connection Error | Permission Error | Undefined Error |
+---------+------------------+------------------+-----------------+
|  False  |      False       |      False       |      False      |
+---------+------------------+------------------+-----------------+
+---------+-------------+-----------+------------+-------+-----------------+--------+-------+-------+--------+
| Address | Compromised | Reachable | Discovered | Value | Discovery Value | Access | linux |  ssh  | tomcat |
+---------+-------------+-----------+------------+-------+-----------------+--------+-------+-------+--------+
|  (1, 0) |    False    |    True   |    True    |  0.0  |       0.0       |  0.0   | False | False | False  |
|  (0, 0) |    False    |   False   |   False    |  0.0  |       0.0       |  0.0   | False | False | False  |
|  (0, 0) |    False    |   False   |   False    |  0.0  |       0.0       |  0.0   | False 

In [17]:
#testing rendering for larger/more complex network
# to specify render mode
env6 = gym.make("nasim:MediumPO2DVA-v0", render_mode="human")

env6.reset()
# render the environment
# (if render_mode="human" is not passed during initialization this will do nothing)
env6.render()


Observation:
+---------+------------------+------------------+-----------------+
| Success | Connection Error | Permission Error | Undefined Error |
+---------+------------------+------------------+-----------------+
|  False  |      False       |      False       |      False      |
+---------+------------------+------------------+-----------------+
+---------+-------------+-----------+------------+-------+-----------------+--------+-------+---------+-------+-------+-------+-------+-------+--------+---------+---------+
| Address | Compromised | Reachable | Discovered | Value | Discovery Value | Access | linux | windows |  ssh  |  ftp  |  http | samba |  smtp | tomcat | daclsvc | schtask |
+---------+-------------+-----------+------------+-------+-----------------+--------+-------+---------+-------+-------+-------+-------+-------+--------+---------+---------+
|  (1, 0) |    False    |    True   |    True    |  0.0  |       0.0       |  0.0   | False |  False  | False | False | False | 

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:42: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (17, 27)
  logger.warn(


In [18]:
#The number of actions can be retrieved from the environment action_space attribute:
#flat_actions = true by default

thisenv = nasim.make_benchmark("tiny")
    # When flat_actions=True
num_actions = thisenv.action_space.n

    # When flat_actions=False
#nvec_actions = thisenv.action_space.nvec

#The shape of the observations can be retrieved from the environment observation_space attribute
obs_shape = thisenv.observation_space.shape

#Getting initial observation and resetting the environment using reset() function
#o, info = env.reset()

In [19]:
agent = AnAgent(...)

o, info = env.reset()
total_reward = 0
done = False
step_limit_reached = False
while not done and not step_limit_reached:
    a = agent.choose_action(o)
    o, r, done, step_limit_reached, info = env.step(a)
    total_reward += r

print("Done")
print("Total reward =", total_reward)

NameError: name 'AnAgent' is not defined